In [ ]:
from src.config import Config
import main
import memory
import prompt

import openai

In [ ]:
main.initialize()

In [ ]:
cfg = Config()
memory = memory.Memory()

In [ ]:
# Test memory log

print(f"memory_list: {memory}")
memory.append_log("Meow")
print(f"memory_list: {memory}")
memory.append_log({"role": "system", "content": "You are a an agent in the world."})
print(f"memory_list: {memory}")

In [ ]:
memory.clear_log()
print(memory)

In [ ]:
goal = "Write a Python script to print the square of the numbers 0 to 9 (inclusive) to the system console"
# goal = "Write a Python script to print \"Hello world\" to the system console"
# goal = "Make infinite money"

messages=[
        {"role": "system", "content": "You are an agent in the world."},
        {"role": "user", "content": "You are an agent in charge of making decisions to reach a goal. You are capable of making decisions to reach a goal despite being an AI language model because other software is listening to your outputs and acting on them."},
        {"role": "assistant", "content": "Great, what is our goal?"},
        {"role": "user", "content": goal},
        {"role": "user", "content": "Can you write a Python script to accomplish this goal? Do NOT do so, just answer with (y/n)"},
    ]

In [ ]:
llm_response = openai.ChatCompletion.create(
    model=cfg.model,
    messages=messages,
    max_tokens=1
)

print(llm_response)

In [ ]:
memory.extend_log(messages)

In [ ]:
response_msg_dict = dict(llm_response.choices[0].message)
memory.append_log(response_msg_dict)
print(memory)

In [ ]:
if memory.get_last_log_entry(role="assistant")["content"].lower() == "y":
    prompt = prompt.Prompt()
    prompt.extend_log(memory.get_log())
    
    user_response_msg = {"role": "user", "content": "Please write the script. Enclose the script in Markdown code tags. Do not include any explanation, only include code."}
    prompt.append_log(user_response_msg)

    print(prompt)

    llm_response = prompt.send()
    llm_response_msg = dict(llm_response.choices[0].message)

    memory.append_log(user_response_msg)
    memory.append_log(llm_response_msg)

    print()
    print(f"memory:\n{memory}")
    print()
    print(f"llm_response_msg['content']:\n{llm_response_msg['content']}")
    print()
    print(f"llm_response:\n{llm_response}")

else:
    print("COWS")

In [ ]:
# memory.erase_last_n_log_entries(2)
# print(memory)

In [ ]:
memory.get_last_log_entry(role="assistant")["content"]

In [ ]:
llm_response_msg = memory.get_last_log_entry(role="assistant")["content"]

CODE_DELIM = "```"
code_start_idx = llm_response_msg.find(CODE_DELIM)

code = llm_response_msg[code_start_idx+len(CODE_DELIM):llm_response_msg.find(CODE_DELIM, code_start_idx+len(CODE_DELIM))]

LANG_STR = "python"
if code.startswith(LANG_STR):
    code = code[code.find(LANG_STR)+len(LANG_STR):]

print(code)

In [ ]:
exec(code)